In [ ]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from ISLP import load_data
from ISLP.models import (summarize, poly, ModelSpec as MS)
from matplotlib.pyplot import subplots

Wage = load_data('Wage')
y = Wage['wage']
age = Wage['age']

age_grid = np.linspace(age.min(), age.max(), 100)
age_df = pd.DataFrame({'age': age_grid})

First computational visit.

Scatter plot.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(age, y, alpha=0.35)
plt.xlabel("Age")
plt.ylabel("Wage")
plt.title("Wage vs Age (Wage dataset)")
plt.show()

Step 2. Let's build our linear model.

In [ ]:
X = sm.add_constant(Wage[['age']])   # matrix with intercept
lm = sm.OLS(y, X).fit() # least squares

# new grid
X_new = sm.add_constant(pd.DataFrame({'age': age_grid}))

# least squares on new grid 
y_hat = lm.predict(X_new)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(age, y, alpha=0.25, color = 'lightblue')
plt.plot(age_grid, y_hat, linewidth=3, color ='darkred')
plt.xlabel("Age")
plt.ylabel("Wage")
plt.title("Linear Regression: Wage ~ Age")
plt.show()

How good / bad are we doing here? 

Let's compute some residuals. 

In [ ]:
res = y - lm.fittedvalues # just subtracting 

plt.figure(figsize=(8, 6))
plt.scatter(age, res, alpha=0.15, color = 'darkred')
plt.axhline(0, color='black', linewidth=2)
plt.xlabel("Age")
plt.ylabel("Residual (wage - fitted)")
plt.title("Residuals vs Age (Linear Model)")
plt.show()

There is still some structure in the residuals. We want these noisy and not with any structure.

On to polynomials.

In [ ]:
# Polynomial Excursion 

degrees = [2, 5, 9]

plt.figure(figsize=(8, 6))
plt.scatter(age, y, alpha=0.15, color = 'lightblue')

for d in degrees:
    basis = MS([poly('age', degree=d)]).fit(Wage)
    X = sm.add_constant(basis.transform(Wage))
    X_new = sm.add_constant(basis.transform(age_df), has_constant='add')

    model = sm.OLS(y, X).fit()
    
    X_new_aligned = X_new.reindex(columns=X.columns, fill_value=0)
    y_hat = model.predict(X_new_aligned)

    plt.plot(age_grid, y_hat, linewidth=3, label=f"degree {d}")

plt.xlabel("Age")
plt.ylabel("Wage")
plt.title("Polynomial regression")
plt.legend()
plt.show()


One-dimensional spline

Wage as a function of age only, considering various smoothness penalties 

In [ ]:
from pygam import LinearGAM, s, f

X = Wage[['age']].to_numpy()
Xg = age_grid.reshape(-1, 1)

# Pick a few values spanning orders of magnitude
lams = [0.0001, 0.01, 0.1, 1, 100, 1000, 100000]  

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')

for lam in lams:
    gam = LinearGAM(s(0), lam=lam).fit(X, y)
    yhat = gam.predict(Xg)
    plt.plot(age_grid, yhat, linewidth=3, label=f"$\lambda$={lam:g}")

plt.xlabel("Age")
plt.ylabel("Wage")
plt.title("GAM: wage ~ s(age) with different smoothness ($\lambda$)")
plt.legend()
plt.show()


Compute effective degrees of freedom. This gives us a single number for the wiggliness of a spline. 

In [ ]:
for lam in lams:
    gam = LinearGAM(s(0), lam=lam).fit(X, y)
    print(f"lambda = {lam:g}  ->  effective degrees of freedom = {gam.statistics_['edof']:.2f}")

Let's use a cross-validation grid search to help find an appropriate penalty lambda. For each of the lambdas in our range, how well can we estimate the the data? 

In [ ]:
lams_grid = np.logspace(-5, 5, 150)
gam_cv = LinearGAM(s(0)).gridsearch(X, y, lam=lams_grid)

print("Selected lambda: ", gam_cv.lam)


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')
plt.plot(age_grid, gam_cv.predict(Xg), color='darkred', linewidth=4, label="gridsearch selected")
plt.xlabel("Age"); plt.ylabel("Wage")
plt.title("GAM: wage ~ s(age) ($\lambda$ selected)")
plt.legend()
plt.show()


On to true Generalized Additive Models, first attempting with a default lambda. 

In [ ]:
# GAM: wage vs age, default penalty

X_age = Wage[['age']].to_numpy()
gam_age = LinearGAM(s(0)).fit(X_age, y)
print(gam_age.lam)

Xg = age_grid.reshape(-1, 1)
y_hat = gam_age.predict(Xg)

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')
plt.plot(age_grid, y_hat, color='darkred', linewidth=3)
plt.xlabel("Age"); plt.ylabel("Wage")
plt.title("GAM: wage ~ s(age)")
plt.show()


Let's do another gridsearch to find an appropriate penalty. 

In [ ]:
# GAM: wage vs age, gridsearch "optimized" lambda

lams_grid = np.logspace(-5, 5, 150)

gam_age_cv = LinearGAM(s(0)).gridsearch(X_age, y, lam=lams_grid)
yhat_cv = gam_age_cv.predict(Xg)

print("Gridsearch-selected lam:", gam_age_cv.lam)
print("Gridsearch edof:", gam_age_cv.statistics_["edof"])

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')
plt.plot(age_grid, yhat_cv, color='darkred', linewidth=3)
plt.xlabel("Age"); plt.ylabel("Wage")
plt.title("GAM (gridsearch $\lambda$): wage ~ s(age)")
plt.show()

Using the 1-D (age only) GAM, how much structure are we leaving out?

In [ ]:
resid_gam = y - gam_age_cv.predict(X_age)

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], resid_gam, alpha=0.25, color='darkred')
plt.axhline(0, color='black', linewidth=2)
plt.xlabel("Age"); plt.ylabel("Residual (wage - fitted)")
plt.title("Residuals vs Age (GAM: s(age))")
plt.show()


This is our first real GAM (age and year). Let's compute a penalty term and see how well we do.

In [ ]:
# fit age year 

from pygam import LinearGAM, s

X_ay = Wage[['age', 'year']].to_numpy()
year0 = float(Wage['year'].mean()) #mean year

lams_grid2 = np.logspace(-5, 5, 100)
gam_ay_cv = LinearGAM(s(0) + s(1)).gridsearch(X_ay, y, lam=lams_grid2)
print("Selected lam (age+year):", gam_ay_cv.lam)
print("edof (age+year):", gam_ay_cv.statistics_["edof"])

Xg_ay = np.column_stack([age_grid,np.full_like(age_grid, year0)])

yhat_ay = gam_ay_cv.predict(Xg_ay)

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')
plt.plot(age_grid, yhat_ay, color='darkred', linewidth=3)
plt.xlabel("Age"); plt.ylabel("Wage")
plt.title(f"GAM (gridsearch $\lambda$): wage ~ s(age) + s(year)  (year fixed at {year0:.1f})")
plt.show()

Plot the partial effect of age and year. 

Partial effect is a tremendous tool for our interpretation of the GAM. As we include predictors, what is the effect? 

Given a GAM of wage = b0 + f1(age) + f2(year), how much does age contribute? 

In [ ]:
# plot the partial effect of age and year

# Partial effect of age
XX_age = gam_ay_cv.generate_X_grid(term=0)
p_age = gam_ay_cv.partial_dependence(term=0, X=XX_age)

plt.figure(figsize=(8, 6))
plt.plot(XX_age[:, 0], p_age, color='black', linewidth=3)
plt.xlabel("Age"); plt.ylabel("Partial effect")
plt.title("Partial effect: s(age)  (from wage ~ s(age) + s(year))")
plt.show()

# Partial effect of year
XX_year = gam_ay_cv.generate_X_grid(term=1)
p_year = gam_ay_cv.partial_dependence(term=1, X=XX_year)

plt.figure(figsize=(8, 8))
plt.plot(XX_year[:, 1], p_year, color='black', linewidth=3)
plt.xlabel("Year"); plt.ylabel("Partial effect")
plt.title("Partial effect: s(year)  (from wage ~ s(age) + s(year))")
plt.show()


Let's finish up. Include education in our Generalized Additive Model. 
Consider the partial dependence. 

In [ ]:
# full model with age + year + education 

edu = Wage['education'].astype('category')
edu_codes = edu.cat.codes.to_numpy()

X_aye = np.column_stack([Wage['age'].to_numpy(),Wage['year'].to_numpy(),edu_codes])

# Same 1D lam grid applied to all terms (keeps it simple for a live demo)
lams_grid3 = np.logspace(-3, 3, 150)
gam_aye_cv = LinearGAM(s(0) + s(1) + f(2)).gridsearch(X_aye, y, lam=lams_grid3)

print("Selected lambda (age + year + edu):", gam_aye_cv.lam)
print("edof (age + year + edu):", gam_aye_cv.statistics_["edof"])

XX_age2 = gam_aye_cv.generate_X_grid(term=0)
p_age2 = gam_aye_cv.partial_dependence(term=0, X=XX_age2)

plt.figure(figsize=(8, 6))
plt.plot(XX_age2[:, 0], p_age2, color='darkred', linewidth=3)
plt.xlabel("Age"); plt.ylabel("Partial effect")
plt.title("Partial effect: s(age)  (full model: s(age) + s(year) + f(education))")
plt.show()


Given the shift from Education, for a mean year, let's look at our model. 

In [ ]:
#final plot - wage vs age, fixed mean year, each education category 

year0 = float(Wage['year'].mean())

plt.figure(figsize=(8, 6))
plt.scatter(Wage['age'], y, alpha=0.25, color = 'lightblue')

for code, lvl in enumerate(edu.cat.categories):
    Xg = np.column_stack([age_grid, np.full_like(age_grid, year0), np.full_like(age_grid, code)])
    yhat = gam_aye_cv.predict(Xg)
    plt.plot(age_grid, yhat, linewidth=3, label=str(lvl))

plt.xlabel("Age"); plt.ylabel("Wage")
plt.title(f"Predicted wage vs age by education (year fixed at mean={year0:.1f})")
plt.legend(title="Education", fontsize=9)
plt.show()